In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import Optimus

In [4]:
# Create optimus
op = Optimus()

In [5]:
df = op.read.csv("random.csv",header=True, sep=";")

## Key Collision

In [132]:
from optimus.ml.Keycollision import KeyCollision
op.spark.conf.set("spark.sql.shuffle.partitions", "4")
op.spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 1 * 1024 * 1024 * 1024)

keyCol = KeyCollision()
keyCol.fingerprint(df, 'STATE').table(columns=["STATE","STATE_fingerprint"])

Using 'column_exp' to process column 'STATE_fingerprint' with function _trim
Using 'column_exp' to process column 'STATE_fingerprint' with function _lower
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _remove_special_chars
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _remove_accents
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _split_sort_remove_join


STATE (string),STATE_fingerprint (string)
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal
Distrito⸱Federal,distritofederal


In [133]:
keyCol.fingerprint_cluster(df, 'STATE').table()

Using 'column_exp' to process column 'STATE_fingerprint' with function _trim
Using 'column_exp' to process column 'STATE_fingerprint' with function _lower
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _remove_special_chars
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _remove_accents
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _split_sort_remove_join


cluster_size (int),cluster (array<string>),count (bigint),recommended (string)
2,"['México⸱D.F.',⸱'Mexico⸱D.F.']",2495,México⸱D.F.
1,['Distriro⸱Federal'],259,Distriro⸱Federal
3,"['Distrito⸱Federal',⸱'DISTRITO⸱FEDERAL',⸱'distrito⸱federal']",11930,Distrito⸱Federal
1,['D.F.'],66,D.F.
1,['Estado⸱de⸱México'],810,Estado⸱de⸱México


In [134]:
keyCol.n_gram_fingerprint(df, 'STATE', 2).table(columns=["STATE","STATE_ngram", "STATE_ngram_fingerprint"])

Using 'column_exp' to process column 'STATE_ngram' with function _lower
Using 'column_exp' to process column 'STATE_ngram' with function _remove_white_spaces
Using 'pandas_udf' to process column 'STATE_ngram' with function _remove_special_chars
Using 'pandas_udf' to process column 'STATE_ngram' with function _remove_accents
Using 'column_exp' to process column 'STATE_ngram' with function _split
Using 'pandas_udf' to process column 'STATE_ngram_fingerprint' with function remote_white_spaces_remove_sort_join


STATE (string),STATE_ngram (array<string>),STATE_ngram_fingerprint (string)
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr
Distrito⸱Federal,"['d',⸱'i',⸱'s',⸱'t',⸱'r',⸱'i',⸱'t',⸱'o',⸱'f',⸱'e',⸱'d',⸱'e',⸱'r',⸱'a',⸱'l',⸱'']",aldediederfeisitlofraristtotr


In [135]:
keyCol.n_gram_fingerprint_cluster(df, "STATE" , 2).table()

Using 'column_exp' to process column 'STATE_ngram' with function _lower
Using 'column_exp' to process column 'STATE_ngram' with function _remove_white_spaces
Using 'pandas_udf' to process column 'STATE_ngram' with function _remove_special_chars
Using 'pandas_udf' to process column 'STATE_ngram' with function _remove_accents
Using 'column_exp' to process column 'STATE_ngram' with function _split
Using 'pandas_udf' to process column 'STATE_ngram_fingerprint' with function remote_white_spaces_remove_sort_join


cluster_size (int),cluster (array<string>),count (bigint),recommended (string)
2,"['México⸱D.F.',⸱'Mexico⸱D.F.']",2495,México⸱D.F.
1,['D.F.'],66,D.F.
3,"['Distrito⸱Federal',⸱'DISTRITO⸱FEDERAL',⸱'distrito⸱federal']",11930,Distrito⸱Federal
1,['Estado⸱de⸱México'],810,Estado⸱de⸱México
1,['Distriro⸱Federal'],259,Distriro⸱Federal


In [136]:
from optimus.ml.DistanceCluster import DistanceCluster as dc
dc.levenshtein_matrix(df,"STATE").table()

Using 'column_exp' to process column 'STATE_fingerprint' with function _trim
Using 'column_exp' to process column 'STATE_fingerprint' with function _lower
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _remove_special_chars
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _remove_accents
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _split_sort_remove_join


STATE_LEVENSHTEIN_1 (string),STATE_LEVENSHTEIN_2 (string),STATE_LEVENSHTEIN_DISTANCE (int)
mexicodf,mexicodf,0
mexicodf,distrirofederal,12
mexicodf,distritofederal,12
mexicodf,df,6
mexicodf,estadodemexico,10
distrirofederal,mexicodf,12
distrirofederal,distrirofederal,0
distrirofederal,distritofederal,1
distrirofederal,df,13
distrirofederal,estadodemexico,11


In [137]:
from optimus.ml.DistanceCluster import DistanceCluster as dc
dc.levenshtein_filter(df,"STATE").table()

Using 'column_exp' to process column 'STATE_fingerprint' with function _trim
Using 'column_exp' to process column 'STATE_fingerprint' with function _lower
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _remove_special_chars
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _remove_accents
Using 'pandas_udf' to process column 'STATE_fingerprint' with function _split_sort_remove_join


STATE_FROM (string),STATE_LEVENSHTEIN_DISTANCE (int),STATE_TO (string)
df,6,mexicodf
mexicodf,6,df
distrirofederal,1,distritofederal
distritofederal,1,distrirofederal
estadodemexico,10,mexicodf


In [205]:
df_l = dc.levenshtein_cluster(df,"STATE")

Using 'column_exp' to process column 'STATE_FINGERPRINT' with function _trim
Using 'column_exp' to process column 'STATE_FINGERPRINT' with function _lower
Using 'pandas_udf' to process column 'STATE_FINGERPRINT' with function _remove_special_chars
Using 'pandas_udf' to process column 'STATE_FINGERPRINT' with function _remove_accents
Using 'pandas_udf' to process column 'STATE_FINGERPRINT' with function _split_sort_remove_join
Using 'column_exp' to process column 'STATE_FINGERPRINT' with function _trim
Using 'column_exp' to process column 'STATE_FINGERPRINT' with function _lower
Using 'pandas_udf' to process column 'STATE_FINGERPRINT' with function _remove_special_chars
Using 'pandas_udf' to process column 'STATE_FINGERPRINT' with function _remove_accents
Using 'pandas_udf' to process column 'STATE_FINGERPRINT' with function _split_sort_remove_join


cluster (array<string>),cluster_size (int),recommended (string),count (bigint)
['D.F.'],1,D.F.,66
"['México⸱D.F.',⸱'Mexico⸱D.F.']",2,México⸱D.F.,2495
['Distriro⸱Federal'],1,Distriro⸱Federal,259
"['Distrito⸱Federal',⸱'DISTRITO⸱FEDERAL',⸱'distrito⸱federal']",3,Distrito⸱Federal,11930
['Estado⸱de⸱México'],1,Estado⸱de⸱México,810


## Feature Engineering

In [15]:
data = [('Japan', 'Tokyo', 37800000),('USA', 'New York', 19795791),('France', 'Paris', 12341418),
              ('Spain','Madrid',6489162)]
df = op.spark.createDataFrame(data, ["country", "city", "population"])

In [16]:
df.table()

country (string),city (string),population (bigint)
Japan,Tokyo,37800000
USA,New⸱York,19795791
France,Paris,12341418
Spain,Madrid,6489162


In [6]:
import optimus.ml.feature as fe

### String to Index

In [18]:
df_sti = fe.string_to_index(df, input_cols=["city", "country"])

In [19]:
df_sti.table()

country (string),city (string),population (bigint),city_index (double),country_index (double)
Japan,Tokyo,37800000,1.0,1.0
USA,New⸱York,19795791,2.0,3.0
France,Paris,12341418,3.0,2.0
Spain,Madrid,6489162,0.0,0.0


### Index to string

In [20]:
# Going back to strings from index
df_its = fe.index_to_string(df_sti, input_cols=["country_index"])

# Show DF with column "county_index" back to string
df_its.table()

country (string),city (string),population (bigint),city_index (double),country_index (double),country_index_string (string)
Japan,Tokyo,37800000,1.0,1.0,Japan
USA,New⸱York,19795791,2.0,3.0,USA
France,Paris,12341418,3.0,2.0,France
Spain,Madrid,6489162,0.0,0.0,Spain


### One Hot Enconder

In [21]:
# Creating DataFrame
data = [
(0, "a"),
(1, "b"),
(2, "c"),
(3, "a"),
(4, "a"),
(5, "c")
]
df = op.spark.createDataFrame(data,["id", "category"])

# One Hot Encoding
df_ohe = fe.one_hot_encoder(df, input_cols=["id"])

# Show encoded dataframe
df_ohe.table()

id (bigint),category (string),id_encoded (vector)
0,a,"(5,[0],[1.0])"
1,b,"(5,[1],[1.0])"
2,c,"(5,[2],[1.0])"
3,a,"(5,[3],[1.0])"
4,a,"(5,[4],[1.0])"
5,c,"(5,[],[])"


### Vector assembler

In [25]:
# Import Vectors
from pyspark.ml.linalg import Vectors

# Creating DataFrame
data = [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)]

df = op.spark.createDataFrame(data,["id", "hour", "mobile", "user_features", "clicked"])

# Assemble features
df_va = fe.vector_assembler(df, input_cols=["hour", "mobile", "user_features"])


# Show assembled df
print("Assembled columns 'hour', 'mobile', 'user_features' to vector column 'features'")
df_va.select("features", "clicked").table()

Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'


features (vector),clicked (double)
"[18.0,1.0,0.0,10.0,0.5]",1.0


### Normalizer

In [8]:
# Import Vectors
from pyspark.ml.linalg import Vectors

data = [
(0, Vectors.dense([1.0, 0.5, -1.0]),),
(1, Vectors.dense([2.0, 1.0, 1.0]),),
(2, Vectors.dense([4.0, 10.0, 2.0]),)
]

df = op.spark.createDataFrame(data,["id", "features"])

df_norm = fe.normalizer(df, input_cols=["features"], p=2.0)

df_norm.table()

id (bigint),features (vector),features_normalized (vector)
0,"[1.0,0.5,-1.0]","[0.666666666667,0.333333333333,-0.666666666667]"
1,"[2.0,1.0,1.0]","[0.816496580928,0.408248290464,0.408248290464]"
2,"[4.0,10.0,2.0]","[0.36514837167,0.912870929175,0.182574185835]"
